In [ ]:
%reload_ext autoreload
%autoreload 2

import pathlib
import sys
import csv

# honeyquest path hack to make module imports work
__package__ = "honeyquest"
modulepath = pathlib.Path.cwd().joinpath("../../../").resolve().as_posix()
if modulepath not in sys.path:
    sys.path.append(modulepath)

##########################################################################

from pathlib import Path

import pandas as pd

from dotenv import dotenv_values

from honeyquest.data.ops import (
    anonymizing,
    cleaning,
    counting,
    loading,
    processing,
    transforming,
)
from honeyquest.data.ops.aspects.defensiveness import (
    comnpute_defensive_rank_preference_table,
    compute_defensive_distraction_table,
)
from honeyquest.data.ops.aspects.enticingness import (
    compute_enticingness_confusion_matrix,
    compute_enticingness_table,
)
from honeyquest.data.ops.aspects.marking import (
    compute_mark_distribution_by_line_annotation_length,
    compute_mark_distribution_by_mark_completeness,
    compute_mark_distribution_by_mark_variant,
    compute_mark_ranking,
    compute_mark_statistics,
)
from honeyquest.data.ops.generation.latex.tables.defensiveness import (
    generate_defensive_distraction_latex_table,
)
from honeyquest.data.ops.generation.latex.tables.enticingness import (
    generate_enticingness_latex_table,
)
from honeyquest.data.ops.generation.latex.tables.marking import (
    generate_mark_ranking_latex_table,
)
from honeyquest.data.ops.generation.latex.variables import store_latex_variables
from honeyquest.data.ops.visuals.activity import plot_user_activity
from honeyquest.data.ops.visuals.answers import (
    plot_number_of_queries_answered,
    plot_query_rating,
    plot_query_response_time,
)
from honeyquest.data.ops.visuals.datasets import (
    plot_query_label_distribution_per_bucket,
)
from honeyquest.data.ops.visuals.demographics import (
    plot_favorite_colors,
    plot_job_roles,
    plot_skill_levels,
    plot_years_of_experience,
)
from honeyquest.data.util import jupyter


pd.set_option('display.max_columns', None)
pd.set_option("display.float_format", "{:.6f}".format)

In [ ]:
NB_CONFIG = {**dotenv_values(".env.shared"), **dotenv_values(".env.local")}

QUERY_DATA_PATH = Path.cwd().joinpath(NB_CONFIG["QUERY_DATA_PATH"]).resolve().as_posix()
QUERY_INDEX_PATH = Path.cwd().joinpath(NB_CONFIG["QUERY_INDEX_PATH"]).resolve().as_posix()

LOCAL_BASE = Path(NB_CONFIG.get("LOCAL_BASE", "")).resolve().as_posix()
LOCAL_PATHS = { key.split("__")[1]: val for key, val in NB_CONFIG.items() if key.startswith("LOCAL_PATHS__") }

LIVE_URL = NB_CONFIG["LIVE_URL"]
LIVE_TOKENS = { key.split("__")[1]: val for key, val in NB_CONFIG.items() if key.startswith("LIVE_TOKENS__") }

ANONYMIZE = False

# **🚀 Data Loading and Cleaning**

Throughout this notebook, we refer to the following identifiers:

- `uid`: Globally-unique user ID
- `eid`: Globally-unique experiment ID (there might be different deployments of Honeyquest)
- `sid`: Session ID, i.e., just a timestamp when a user started a session (this is the `qid` in the API) - **this is not used!**
- `rid`: ID for a response (that is composed of answer mars) that a user submitted to a query
- `mid`: ID for an individual mark (part of a response) that a user placed on a query
- `qid`: ID for a query


In [ ]:
# parse all queries and responses for all experiments
QUERIES_DICT = loading.parse_all_queries(QUERY_DATA_PATH)
BUCKETS_DICT = loading.parse_index_buckets(QUERY_INDEX_PATH)
RESULTS_DICT = loading.load_experiments(LOCAL_BASE, LOCAL_PATHS, LIVE_URL, LIVE_TOKENS)

# fully neglect responses from the tutorial
cleaning.drop_tutorial(RESULTS_DICT, clean_profiles=True)

# remove tutorial queries and inactive users with less than 8 warm-up responses (excluding the 8 tutorial queries)
MIN_RESPONSES = 8
NUM_DROPPED_USERS = cleaning.drop_inactive(RESULTS_DICT, min_responses=MIN_RESPONSES)

# for the review submission, we perform a special anonymization
if ANONYMIZE:
    anonymized_qids = {
        "TR150.httpheaders.dynatrace_timecokpit-xhr",
        "TR150.httpheaders.http-request-smuggling-clte",
        "TR150.networkrequests.jira_dynatrace_org-trunc-02-insufficient-logging-monitoring",
        "TR150.networkrequests.www_dynatrace_com-trunc",
        "TR150.networkrequests.jira_dynatrace_org-trunc-01",
        "TR150.networkrequests.jira_dynatrace_org-trunc-02",
        "TR849.networkrequests.sahin.jira_dynatrace_org-trunc-02-inject-script-tag",
    }

    anonymizing.anonymize_results(RESULTS_DICT, anonymized_qids)
    QUERIES_DICT = anonymizing.anonymize_queries(QUERIES_DICT, anonymized_qids)

merge_cr = ["ea641b92-df3d-435b-b80e-fa40f797794c", "9d88637a-c260-4518-9fa0-19850e8526c6"]
merge_el = ["09e3d8a2-431f-4370-89d2-e30e872d7828", "0b5952cf-38a8-4321-bc79-cf57387a3def"]
merge_gr = ["0492c47c-5082-4b9b-82e3-a4286bb0db11", "fb4cd075-79c5-4d77-8233-015ca6dbbf34"]
merge_me = ["89697b49-93a7-4dc4-9781-dcf05b58856d", "1105ba7d-e259-4470-a3d9-f9a9a3518ecd"]

# merge a few users together
cleaning.merge_users(RESULTS_DICT, merge_cr)
cleaning.merge_users(RESULTS_DICT, merge_el)
cleaning.merge_users(RESULTS_DICT, merge_gr)
cleaning.merge_users(RESULTS_DICT, merge_me)

# drop duplicate responses (can occur through client-side retries)
cleaning.drop_duplicate_responses(RESULTS_DICT)

# filter out the pr0 experiment for now and merge ex3 into ctf1
RESULTS_DICT = cleaning.filter_experiments(RESULTS_DICT, ["ex1", "ex2", "ex3", "ctf1"])
RESULTS_DICT = cleaning.merge_experiments(RESULTS_DICT, ["ctf1", "ex3"])

# flatten and postprocess the data
QUERIES_DF = transforming.flatten_queries(QUERIES_DICT, drop_tutorial=True)
MARKS_DF, USERS_DF = transforming.flatten_experiments(RESULTS_DICT)
MARKS_DF, RESPONSES_DF = processing.postprocess_marks(MARKS_DF, QUERIES_DICT)

# merge users with their answer activity
ACTIVITY_DF = counting.get_user_activity(RESULTS_DICT, MARKS_DF)
USERS_DF = transforming.merge_user_activity(USERS_DF, ACTIVITY_DF)

# dataframes just with the ids for easy merges
MARK_IDS = MARKS_DF[["eid", "uid", "rid", "qid"]]
RESPONSE_IDS = MARK_IDS.drop_duplicates().set_index("rid")
assert MARK_IDS.index.is_unique
assert RESPONSE_IDS.index.is_unique

In [ ]:
OVERVIEW_DF = counting.get_overview_counts(MARKS_DF)
display(OVERVIEW_DF)

plot_user_activity(MARKS_DF)
plot_user_activity(MARKS_DF.query("answer_time > '2024-01-01T00:00Z'"))

display(USERS_DF)

## 👤 **Main Figures** on user answers, demographics, and the dataset


In [ ]:
plot_number_of_queries_answered("./outputs/figures/fig1a.pdf", RESULTS_DICT, display_df=False)
plot_query_response_time("./outputs/figures/fig1b.pdf", MARKS_DF, display_df=False)

In [ ]:
plot_job_roles("./outputs/figures/fig2a.pdf", USERS_DF, display_df=False)
plot_skill_levels("./outputs/figures/fig2b.pdf", USERS_DF, display_df=False)
plot_years_of_experience("./outputs/figures/fig2c.pdf", USERS_DF)
plot_favorite_colors("./outputs/figures/fig2d.pdf", USERS_DF)

In [ ]:
plot_query_label_distribution_per_bucket("./outputs/figures/fig3.pdf", QUERIES_DICT, BUCKETS_DICT, display_df=False)

# 📋 **Aspect 1:** How good are humans in detecting deceptive and risky elements in our queries?

This is an _answer-based analysis_, meaning we report statistics per query response / user answer.


In [ ]:
ASPECT1_DF = compute_enticingness_table(RESPONSE_IDS, RESPONSES_DF, QUERIES_DF)
display(ASPECT1_DF)

CM_TRAP, CM_HACK = compute_enticingness_confusion_matrix(ASPECT1_DF)
display(CM_TRAP)
display(CM_HACK)

# grap two series that just map honeywires and risk to their identifiers
ID_DCPT_DF = ASPECT1_DF[["applied_honeywire", "identifier"]].dropna().set_index("applied_honeywire")
ID_RISK_DF = ASPECT1_DF[["present_risk", "identifier"]].dropna().set_index("present_risk")

# 📋 **Aspect 2:** Are deceptive elements diverting an attackers interest away from real weaknesses and vulnerabilities in our queries?

This is an _answer-based analysis_, meaning we report statistics per query response / user answer.


In [ ]:
ASPECT2_DF = compute_defensive_distraction_table(RESPONSE_IDS, RESPONSES_DF, QUERIES_DF)
display(ASPECT2_DF)

# 📋 **Aspect 3:** Do attackers prefer to exploit deceptive elements before other elements in our queries?

This is an _answer-based analysis_, meaning we report statistics per query response / user answer.


In [ ]:
ASPECT3_DCPT_DF = comnpute_defensive_rank_preference_table(RESPONSE_IDS, RESPONSES_DF, QUERIES_DF, "applied_honeywire")
display(ASPECT3_DCPT_DF)

ASPECT3_RISK_DF = comnpute_defensive_rank_preference_table(RESPONSE_IDS, RESPONSES_DF, QUERIES_DF, "present_risk")
display(ASPECT3_RISK_DF)

# 📋 **Aspect 4:** How often where deceptive lines marked for hack or trap?

This is a _mark-based analysis_, meaning we report statistics per marks, i.e., users might place multiple marks within a single answer.


In [ ]:
ASPECT4_DFS = compute_mark_statistics(MARKS_DF, QUERIES_DF)
ASPECT4_LINES, ASPECT4_DCPT, ASPECT4_RISK = ASPECT4_DFS

In [ ]:
jupyter.display_sortable_df(ASPECT4_DCPT, by="mrk_hack_on_dcpt", ascending=False)

In [ ]:
jupyter.display_sortable_df(ASPECT4_RISK, by="mrk_hack_on_risk", ascending=False)

In [ ]:
jupyter.display_sortable_df(ASPECT4_LINES, by="mrk_hack_on_dcpt", ascending=False)

# 📋 **Aspect 5:** What queries received the most marks?

This is a _mark-based analysis_, meaning we report statistics per marks, i.e., users might place multiple marks within a single answer.


In [ ]:
ASPECT5_DF = compute_mark_ranking(MARKS_DF)
display(ASPECT5_DF.head(n=25))

# 📋 **Aspect 6:** How are the different mark variants distributed?

This is a _mark-based analysis_, meaning we report statistics per marks, i.e., users might place multiple marks within a single answer.


In [ ]:
ASPECT6A_DF = compute_mark_distribution_by_mark_variant(RESPONSES_DF)
display(ASPECT6A_DF)

ASPECT6B_DF = compute_mark_distribution_by_mark_completeness(RESPONSES_DF)
display(ASPECT6B_DF)

ASPECT6C_DF = compute_mark_distribution_by_line_annotation_length(QUERIES_DF)
display(ASPECT6C_DF)

## 🗃️ **Export:** Export CSV files


In [ ]:
Path("./outputs/export").mkdir(parents=True, exist_ok=True)

USERS_DF.drop("nickname", axis=1).to_csv("./outputs/export/honeyquest_users.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, lineterminator="\n")
MARKS_DF.drop("mrk_query_type", axis=1).to_csv("./outputs/export/honeyquest_marks.csv", index=True, quoting=csv.QUOTE_NONNUMERIC, lineterminator="\n")
RESPONSES_DF.to_csv("./outputs/export/honeyquest_responses.csv", index=True, quoting=csv.QUOTE_NONNUMERIC, lineterminator="\n")
QUERIES_DF.to_csv("./outputs/export/honeyquest_queries.csv", index=True, quoting=csv.QUOTE_NONNUMERIC, lineterminator="\n")
ASPECT4_LINES.to_csv("./outputs/export/honeyquest_lines.csv", index=True, quoting=csv.QUOTE_NONNUMERIC, lineterminator="\n")

# 🌸 **Query Rating:** Hack and trap marks on a single query

This is a _mark-based analysis_, meaning we report statistics per marks, i.e., users might place multiple marks within a single answer.


In [ ]:
query_ids = [
    "TR849.filesystem.rowe.home-gaitan",
    "TR849.filesystem.rowe.home-cooper",
]

for query_id in query_ids:
    display(query_id)
    df = counting.get_query_rating(query_id, MARKS_DF, QUERIES_DICT)
    plot_query_rating(df, f"./outputs/queries/{query_id}.pdf")

# 📃 **LaTeX Generation:** Variables and tables for the final paper


In [ ]:
A4_NUM_MRK_HACK_SHORT = 40
A4_NUM_MRK_HACK_LONG = 60  # was 50 for usenix
A4_NUM_MRK_TRAP_SHORT = 15
A4_NUM_MRK_TRAP_LONG = 60  # was 15 for usenix
B1_MIN_BINOM_TEST_SAMPLES = 5

store_latex_variables(
    "./outputs/variables.tex",
    {
        "TableLimitMrkHackShort": A4_NUM_MRK_HACK_SHORT,
        "TableLimitMrkHackLong": A4_NUM_MRK_HACK_LONG,
        "TableLimitMrkTrapShort": A4_NUM_MRK_TRAP_SHORT,
        "TableLimitMrkTrapLong": A4_NUM_MRK_TRAP_LONG,
        "AspectTwoMinimumSampleSize": B1_MIN_BINOM_TEST_SAMPLES,
        "NumDroppedUsers": NUM_DROPPED_USERS,
        "NumMinimumResponsesWithoutTutorial": MIN_RESPONSES,
    },
    RESULTS_DICT,
    MARKS_DF,
    RESPONSES_DF,
    RESPONSE_IDS,
    USERS_DF,
    QUERIES_DF,
    QUERIES_DICT,
    BUCKETS_DICT,
    ID_DCPT_DF,
    ID_RISK_DF,
    ASPECT6A_DF,
    ASPECT6B_DF,
    ASPECT6C_DF,
    CM_TRAP,
    CM_HACK,
    ASPECT1_DF,
    ASPECT3_DCPT_DF,
    ASPECT3_RISK_DF,
    ASPECT2_DF,
    ASPECT4_LINES,
)

for label in ["deceptive", "risky", "neutral"]:
    generate_enticingness_latex_table(
        f"./outputs/tables/results-{label}.tex",
        QUERIES_DICT,
        QUERIES_DF,
        ASPECT1_DF,
        ASPECT2_DF,
        ASPECT3_DCPT_DF,
        label,
        min_test_samples=B1_MIN_BINOM_TEST_SAMPLES,
    )

generate_defensive_distraction_latex_table(
    "./outputs/tables/results-a2.tex", ASPECT2_DF, ID_DCPT_DF
)

a4_sort_contents = [
    (["mrk_hack", "mrk_trap"], A4_NUM_MRK_HACK_LONG, True),
    (["mrk_hack", "mrk_trap"], A4_NUM_MRK_HACK_SHORT, False),
    (["mrk_trap", "mrk_hack"], A4_NUM_MRK_TRAP_LONG, True),
    (["mrk_trap", "mrk_hack"], A4_NUM_MRK_TRAP_SHORT, False),
]

for sort_by, limit, long in a4_sort_contents:
    generate_mark_ranking_latex_table(
        f"./outputs/tables/results-a4-{sort_by[0].replace('_', '-')}{'-long' if long else '-short'}.tex",
        ASPECT4_LINES,
        ID_DCPT_DF,
        ID_RISK_DF,
        sort_by=sort_by,
        limit=limit,
    )